In [1]:
import requests 

In [2]:
import docx
import spacy
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity

# Load spaCy Russian tokenizer
nlp = spacy.blank("ru")

# Load the Ruberta model and tokenizer from Hugging Face transformers
model_name = "cointegrated/rubert-tiny2"  # Pre-trained Ruberta model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def read_docx(file_path):
    # Open the Word document
    doc = docx.Document(file_path)
    
    # Extract text content from the document
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    
    # Join all paragraphs into a single string
    document_text = "\n".join(full_text)
    return document_text

def chunk_and_encode_document(document_text, max_chunk_len=600):
    # Split the document into chunks
    doc = nlp(document_text)
    chunks = []
    current_chunk = []

    for token in doc:
        if len(current_chunk) + len(token.text) > max_chunk_len:
            chunks.append(" ".join(current_chunk))
            current_chunk = []

        current_chunk.append(token.text)

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    # Encode each chunk into vectors
    encoded_chunks = []

    for chunk in chunks:
        # Tokenize the chunk
        inputs = tokenizer(chunk, return_tensors="pt", padding=True, truncation=True)

        # Pass the input through the model and get the hidden states
        with torch.no_grad():
            outputs = model(**inputs)

        # Extract the embeddings (CLS token embedding)
        embeddings = outputs.last_hidden_state[:, 0, :]

        # Normalize the embeddings
        normalized_embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)

        # Append the normalized embedding to the list
        encoded_chunks.append(normalized_embeddings)

    return chunks, encoded_chunks


file_path = "main_doc.docx"
document_text = read_docx(file_path)
chunks, encoded_chunks = chunk_and_encode_document(document_text)

In [3]:
def encode_query(query_text):
    # Encode the query text into a vector
    inputs = tokenizer(query_text, return_tensors="pt", padding=True, truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)

    # Extract the embeddings (CLS token embedding)
    query_embedding = outputs.last_hidden_state[:, 0, :]

    # Normalize the embedding
    normalized_query_embedding = torch.nn.functional.normalize(query_embedding, p=2, dim=1)

    return normalized_query_embedding


import numpy as np
from sklearn.preprocessing import binarize


def euclidean_distance(a, b):
    return np.linalg.norm(a - b)

def vector_search(query_embedding, encoded_chunks, chunks):
    distances = []

    for emb in encoded_chunks:
        dist = euclidean_distance(query_embedding, emb)
        distances.append(dist)

    # Find indices of the top 3 closest chunks (smallest distances)
    closest_indices = np.argsort(distances)[:3]
    closest_chunks = [chunks[idx] for idx in closest_indices]

    return closest_chunks



query = 'За что отвечает академический руководитель?'
query_embedding = encode_query(query)
closest_chunks = vector_search(query_embedding, encoded_chunks, chunks)
i=1
print("Closest chunks to the query:")
for idx, chunk in enumerate(closest_chunks, start=1):
    print('chunk ',i)
    print()
    i+=1
    print(f"{idx}. {chunk}")
    print()

Closest chunks to the query:
chunk  1

1. Экзамена на основании личного заявления , поданного в Центр сервиса « Студент » при предъявлении оригинала сертификата не позднее , чем за 5 рабочих дней до даты начала сессии 3 модуля 3 курса . Этим студентам в качестве оценки за Экзамен выставляется общая оценка из сертификата , переведенная в баллы согласно Шкале соответствия . 
 Студенты , представившие сертификат позднее , чем за 5 рабочих дней до даты начала сессии 3 модуля 3 курса и не пришедшие на Экзамен , считаются имеющими академическую задолженность , порядок ликвидации которой определен в Положении . 

 Особенности проведения Экзамена в дистанционном формате 
 В назначенное время начала Экзамена студент подключается к видеоконференции по ссылке , направленной студенту на корпоративную почту в письме - напоминании накануне даты сдачи Экзамена сотрудниками Центра . 
 В течение всего Экзамена студент обязан присутствовать с включенной камерой в помещении с достаточной для идентификаци

In [7]:
context = str(closest_chunks)

prompt = {
    "modelUri": "gpt://folder-id/yandexgpt-lite",
    "completionOptions": {
        "stream": False,
        "temperature": 0.6,
        "maxTokens": "2000"
    },
    "messages": [
        {
            "role": "system",
            "text": "ты умный помошник студента, используй документы ниже, чтобы ответить на поставленный вопрос, если в документах этой информации нет, то так и скажи \
             вот ДОКУМЕНТЫ: " + context
        },
        {
            "role": "user",
            "text": query
        }
    ]
}


url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
headers = {
    "Content-Type": "application/json",
    "Authorization": "Api-Key your key"
}

response = requests.post(url, headers=headers, json=prompt)
result = response.text
response_data = response.json()
assistant_text = response_data['result']['alternatives'][0]['message']['text']
print(assistant_text)

Согласно приведённому вами документу, **академический руководитель** отвечает за организацию учебного процесса, в том числе за уведомление студентов о пересдачах, организацию пересдач и принятие соответствующих решений.

В документе также указано, что **студенты, получившие неудовлетворительные оценки**, или не явившиеся на внешние экзамены, или не выбравшие курсовую работу или выпускную квалификационную работу (ВКР), уведомляются о необходимости ликвидации академических задолженностей.

Таким образом, на основе приведённого документа можно сделать вывод, что академический руководитель играет важную роль в процессе организации учебного процесса и отвечает за решение академических вопросов, связанных с успеваемостью студентов.
